# Part I. ETL Pipeline for Pre-Processing the Files

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'*'))
    

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
      
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#print(len(full_data_rows_list))
#print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


##  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [5]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_keyspace
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
try:
    session.set_keyspace('music_keyspace')
except Exception as e:
    print(e)


### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


The image below is a screenshot of sample music_playlist table with COMPOSITE PRIMARY KEY as (session_ID,item_In_Session):
<img src="images/Screenshot 2020-04-25 at 11.46.31 AM.png">


In [7]:
query = "CREATE TABLE IF NOT EXISTS music_playlist "
query = query + "(session_ID int,artist text,title text,length double,item_In_Session int,PRIMARY KEY (session_ID,item_In_Session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [8]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO music_playlist (session_ID,artist,title,length,item_In_Session)"
        query = query + "values (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), line[0],line[9],float(line[5]),int(line[3])))

In [9]:
query = "SELECT artist,title,length FROM music_playlist where session_ID = 338 and item_In_Session = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073




### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


The image below is a screenshot of sample song_playlist_session table with 
Partition key : user_ID
Clustering Key : session_ID,item_In_Session
<img src="images/Screenshot 2020-04-25 at 12.02.57 PM.png">


In [10]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(artist text,song text,item_In_Session int,firstname text,lastname text,user_ID int,session_ID int,PRIMARY KEY ((user_ID),session_ID,item_In_Session)) WITH CLUSTERING ORDER BY (session_ID ASC,item_In_Session DESC)"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (artist,song,item_In_Session,firstname,lastname,user_ID,session_ID)"
        query = query + "values (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], line[9],int(line[3]),line[1],line[4],int(line[10]),int(line[8])))

In [12]:
query = "SELECT artist,song,firstname,lastname FROM song_playlist_session where user_ID = 10 and session_ID = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist,row.song,row.firstname,row.lastname)

Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Down To The Bone Keep On Keepin' On Sylvie Cruz


    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


The image below is a screenshot of sample music_playlist table with COMPOSITE PRIMARY KEY as (song,user_ID,firstname,lastname)
    
<img src="images/Screenshot 2020-04-25 at 12.09.12 PM.png">

In [13]:
query = "CREATE TABLE IF NOT EXISTS song_user_list "
query = query + "(song text,user_ID int,firstname text,lastname text,PRIMARY KEY (song,user_ID,firstname,lastname))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user_list (song,user_ID,firstname,lastname)"
        query = query + "values (%s,%s,%s,%s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [15]:
query = "SELECT firstname,lastname from song_user_list where song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname,row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [16]:
query = "drop table music_playlist"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_user_list"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()